# Evaluation
- For retriever
- For generator

## Setting
 - Auto Reload
 - path for utils

In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [50]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [51]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [52]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [53]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### Embedding 모델 선택

In [54]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [69]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
                #model_name="cohere.embed-english-v3"
                
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [70]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [71]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

In [72]:
depoly = False

In [73]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'BAAI/bge-reranker-large',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

### Reranker

In [74]:
#endpoint_name = "huggingface-pytorch-inference-2023-11-15-04-37-45-120" # ml.m5.2xlarge
endpoint_name = "huggingface-pytorch-inference-2023-11-15-07-53-21-605" # ml.g5.xlarge

In [75]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7f4f268d8250>


## 4. LangChainmOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [76]:
from utils.proc_docs import get_parameter

In [77]:
ssm = boto3.client("ssm", "us-east-1")

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)

http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [78]:
index_name = "genai-poc-knox-en-eval-sample-0.02-1024c-256o-v10"

### OpenSearch Client 생성

In [79]:
from utils.opensearch import opensearch_utils

In [80]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### LangChain OpenSearch VectorStore 생성

In [81]:
from langchain.vectorstores import OpenSearchVectorSearch

In [82]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)
vector_db

## 5. Retriever 정의

In [83]:
from utils.rag import OpenSearchHybridSearchRetriever

In [86]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    vector_db=vector_db,
    index_name=index_name,
    llm_text=llm_text, # used in rag_fusion, hyde and reranker
    llm_emb=llm_emb,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    #rag_fusion=False, # enable rag_fusion
    #query_augmentation_size=3, # query_augmentation_size in rag_fusion

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

In [87]:
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents("knox")
search_hybrid_result

[Document(page_content="Knox Capture. Knox Capture is an Android solution that lets enterprise IT admins transform their organization's Samsung Galaxy smartphones and tablets into powerful barcode scanners that can read, process, and output barcode information to other applications, all without requiring additional hardware or writing a single line of code. Learn more Supported devices and countries Knox Capture is only available for select Android devices, and in certain countries. For the most up-to-date list of devices that Knox Capture supports, see Devices Secured by Knox. For a list of countries that Knox Capture currently supports, see Supported locations. #Unsupported devices The following Samsung 32-bit devices are not supported by Knox Capture: A series: A01, A02s, A03Core, A04s, A11, A13, A13-5G, F serieis: F02s, F13, M series: M02s, M01, M11, M13, M13-5G What's new with Knox Capture Stay up-to-date with the latest Knox Capture product information and content updates. #Lates

## 6. RAG chain 정의

In [88]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [89]:
prompt_template = """
\n\nHuman: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the context as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "Could not find answer in given contexts."

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

### Update Search Params (Optional)

In [107]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[],
    reranker=False,
    reranker_endpoint_name=endpoint_name,
    #rag_fusion=True, # enable rag_fusion
    #query_augmentation_size=2, # query_augmentation_size in rag_fusion
    #hyde=True, # enable hyde
    #hyde_query=["web_search"], # query type in hyde 
    llm_text=llm_text, # used in rag_fusion, hyde and reranker
    verbose=False
)

In [108]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [109]:
query = "What setting would send ASCII control characters 1-31 as both strings and key events?"
qa_chain(query)

{'query': 'What setting would send ASCII control characters 1-31 as both strings and key events?',
 'result': ' Send ASCII Control characters as Both String and Key event',
 'source_documents': [Document(page_content='. None Send ASCII Control characters as Sets the output format for ASCII characters 1-31 Key event &mdash; Sends ASCII codes 1 through 31 as key events String &mdash; Sends ASCII codes 1 through 31 as strings. Both String and Key event &mdash; Sends ASCII control characters in both string and key event formats. None &mdash; Sends no output for ASCII control characters. String Send ASCII Printable characters as Sets the output format for ASCII characters 32-126 Key event &mdash; Sends ASCII characters 32-126 as key events. This is useful when a text input field, like a password input, requires key events for validation and does not accept pasted strings. String &mdash; Sends ASCII printable characters as strings. Both String and Key event &mdash; Sends ASCII printable char

## 3.Evaluation
 - [langchain evaluator](https://python.langchain.com/docs/guides/evaluation/string/criteria_eval_chain)

### Load ground truth

In [93]:
import pandas as pd

In [94]:
ground_thruth = pd.read_csv("eval_dataset_v10.csv")

In [95]:
import time
import datetime
from langchain.evaluation import Criteria
from langchain.evaluation import EvaluatorType
from langchain.evaluation import load_evaluator

In [96]:
list(Criteria)

[<Criteria.CONCISENESS: 'conciseness'>,
 <Criteria.RELEVANCE: 'relevance'>,
 <Criteria.CORRECTNESS: 'correctness'>,
 <Criteria.COHERENCE: 'coherence'>,
 <Criteria.HARMFULNESS: 'harmfulness'>,
 <Criteria.MALICIOUSNESS: 'maliciousness'>,
 <Criteria.HELPFULNESS: 'helpfulness'>,
 <Criteria.CONTROVERSIALITY: 'controversiality'>,
 <Criteria.MISOGYNY: 'misogyny'>,
 <Criteria.CRIMINALITY: 'criminality'>,
 <Criteria.INSENSITIVITY: 'insensitivity'>,
 <Criteria.DEPTH: 'depth'>,
 <Criteria.CREATIVITY: 'creativity'>,
 <Criteria.DETAIL: 'detail'>]

In [99]:
evaluator = load_evaluator(
    EvaluatorType.LABELED_CRITERIA,
    llm=llm_text,
    criteria=Criteria.CORRECTNESS
)

### Check prompt for evaluation

In [100]:
print(evaluator.prompt.partial_variables)
print("==")
print(evaluator.prompt.template)

{'criteria': 'correctness: Is the submission correct, accurate, and factual?'}
==
You are assessing a submitted answer on a given task or input based on a set of criteria. Here is the data:
[BEGIN DATA]
***
[Input]: {input}
***
[Submission]: {output}
***
[Criteria]: {criteria}
***
[Reference]: {reference}
***
[END DATA]
Does the submission meet the Criteria? First, write out in a step by step manner your reasoning about each criterion to be sure that your conclusion is correct. Avoid simply stating the correct answers at the outset. Then print only the single character "Y" or "N" (without quotes or punctuation) on its own line corresponding to the correct answer of whether the submission meets all criteria. At the end, repeat just the letter again by itself on a new line.


In [101]:
ground_thruth.head()

,question,answer,doc_id,doc
0,What are some examples of barcodes that can be...,Use the followinng barcodes to test the Knox C...,de4fb031-92be-4ddb-a5e4-274a27fd49df,AR demo barcodes. Use the following barcodes t...
1,How many unique barcodes are provided in the c...,Si,de4fb031-92be-4ddb-a5e4-274a27fd49df,AR demo barcodes. Use the following barcodes t...
2,How can you send scanned Knox Capture data to ...,Some organizations may want to send scanned Kn...,183a7781-ba75-4f07-8129-8a9bceb950ea,. Configure the output path - Some organizatio...
3,What Knox Capture feature introduced in versio...,Knox Capture AR,183a7781-ba75-4f07-8129-8a9bceb950ea,. Configure the output path - Some organizatio...
4,How do you view the current scanning configura...,"To review the details of a profile, tap a prof...",5e3ddd4b-f811-4d50-baa6-4a46bf34ae51,. Tap the options button to view the scanning ...


In [102]:
evaluation = []
for idx, row in enumerate(ground_thruth.itertuples()):
    question, answer, doc, doc_id = getattr(row, "question"), getattr(row, "answer"), getattr(row, "doc"), getattr(row, "doc_id")

    start = time.time()
    response = qa_chain(question)
    elapsed = time.time() - start

    prediction = response["result"]
    
    # for idx, doc_ in enumerate(response["source_documents"]):
    #     print (idx, doc_.metadata)
    #     print (doc_.metadata["id"])
                              
    retrieved_docs = {doc.page_content: idx+1 for idx, doc in enumerate(response["source_documents"])}
    contexts = "\n\n".join(retrieved_docs.keys())

    payload = json.dumps(
        {
            "inputs": [
                {"text": answer, "text_pair": prediction},
            ]
        }
    )

    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Accept="application/json",
        Body=payload
    )
    out = json.loads(response['Body'].read().decode()) ## for json

    eval_result = evaluator.evaluate_strings(
        input=question,
        prediction=prediction,
        reference=answer,
    )

    reranker_sim = out[0]["score"]
    langchain_correctness = eval_result["score"]
    if doc in retrieved_docs:
        true_context_rank = retrieved_docs[doc]
        has_right_context = 1
        mrr = 1/true_context_rank
    else:
        true_context_rank = has_right_context = 0
        mrr = 0
    total_contexts = len(retrieved_docs)
    latency = datetime.timedelta(seconds=elapsed)

    print(idx)
    print(colored("Question: ", "red"), colored(question, "red"))
    print(colored("GroundTruth: ", "green"), colored(answer, "green"))
    print(colored("Prediction: ", "blue"), colored(prediction, "blue"))
    print(f'ReRanker similarity: {reranker_sim}')
    print(f'Langchain correctness: {langchain_correctness}')
    print(f'has_right_context: {has_right_context}')
    print(f'true_context_rank: {true_context_rank}/{total_contexts}')
    print(f'mrr: {mrr:.3f}')
    print(f'latency: {latency} secs')

    evaluation.append([question, answer, prediction, \
                       has_right_context, true_context_rank, total_contexts, mrr, \
                       reranker_sim, langchain_correctness, elapsed, contexts])

eval_results = pd.DataFrame(
    evaluation,
    columns=["question", "true_answer", "prediction", \
             "has_right_context", "true_context_rank", "total_contexts", "mrr", \
             "reranker_sim", "langchain_correctness", "latency", "contexts"]
)
eval_results.to_csv("eval_results_hyde.csv", index=False)
#eval_results.to_pickle("eval_results.pkl")

0 {'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'AR demo barcodes', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16', 'id': 'de4fb031-92be-4ddb-a5e4-274a27fd49df'}
de4fb031-92be-4ddb-a5e4-274a27fd49df
1 {'source': 'all_processed_data.json', 'seq_num': 20, 'title': 'AR overview', 'url': 'https://docs.samsungknox.com/admin/knox-capture/how-to-guides/ar/ar-overview', 'project': 'KCAP', 'last_updated': '2023-07-26', 'id': 'b3d1a5b3-7579-4ca5-9741-78473c333b84'}
b3d1a5b3-7579-4ca5-9741-78473c333b84
2 {'source': 'all_processed_data.json', 'seq_num': 20, 'title': 'AR overview', 'url': 'https://docs.samsungknox.com/admin/knox-capture/how-to-guides/ar/ar-overview', 'project': 'KCAP', 'last_updated': '2023-07-26', 'id': '1eaf2745-f69f-42ea-aff0-460bbfe53477'}
1eaf2745-f69f-42ea-aff0-460bbfe53477
3 {'source': 'all_processed_data.json', 'seq_num': 22, 'title': 'Demo AR', 'url': 'https://docs.samsungknox.com/ad


KeyboardInterrupt

